In [154]:
#Imports
import gym
import tensorflow as tf
import numpy as np
import random
from collections import deque 
env = gym.make('CartPole-v1')

In [155]:
#Global varaible
epsilon = 1

In [167]:
#Constants
REPLAY_MEMORY_SIZE = 10000
MIN_REPLAY_MEMORY_SIZE = 250
MIN_EPSILON = 0.01
EPSILON_DECAY = 0.98
BATCH_SIZE = 32
DISCOUNT = 0.9995
UPDATE_PREDICITON_EVERY = 2

In [157]:
#Model takes in actions and outputs Q values for each action which represnt the predicted score for the action
def create_model():
    model = tf.keras.Sequential([tf.keras.layers.Dense(64,input_shape = (4,)),
                                 tf.keras.layers.Dense(32, activation = 'relu'),
                                 tf.keras.layers.Dense(2, activation = 'linear')])
    model.compile(tf.keras.optimizers.Adam(0.001), loss = 'mse', metrics='accuracy')
    return model

In [158]:
#Create new models
updated_model = create_model()

#Prediction model is created to keep some consitancy in future Q values predictions
#This only gets updated after every couple runthroughs of the enviorment
prediction_model = create_model()
prediction_model.set_weights(updated_model.get_weights())

In [203]:
#Returns a action for the Cart Pole enviorment
#If epsilon is bigger there is a larger chance of performing random actions (exploring)
#Epsilon decays over training iterations and actions become more greedy (exploitation)
def get_action(obs):
    if random.uniform(0, 1) <= epsilon:
        return random.randint(0,1)
    else:
        q_vals = updated_model.predict(obs)
        print(q_vals)
        return np.argmax(q_vals)

In [201]:
#Using expirence and past rewards this trains the model to output higher Q values for actions that have better rewards given the state(input)
def train_model():
    #Test to see if it has enough expirence
    if len(replay_memory) >= MIN_REPLAY_MEMORY_SIZE:
        #Get a ranodm batch of samples from memory
        batch = random.sample(replay_memory, BATCH_SIZE)
        
        #X[0] is the intial observations. Getting the Q outputs for the intial enviorment condtions batch
        X = np.array([x[0] for x in batch]).reshape(BATCH_SIZE,4)
        current_qs_list = updated_model.predict(X)
        
        #X[3] is the new observations. Getting the Q outputs for the new enviorment condtions batch
        future_qs_list = prediction_model.predict(np.array([x[3] for x in batch]).reshape(BATCH_SIZE,4))
        
        y = []
        #Loop calculates what the Q output for each sample should look like    
        for i, (obs, action, reward, new_obs, done) in enumerate(batch):
            #This logic allows Q values to predict future Q values
            if not done:                        
                max_future_q = np.max(future_qs_list[i]) 
                new_q = reward + DISCOUNT * max_future_q #Q value for action should be determined from what the best future Q value is when taking the action
            else:
                new_q = reward #If the action caused the enviorment to terminate then the Q value should be the reward it got

            #changing the expected Q value for the action taken in the sample
            current_qs = current_qs_list[i]
            current_qs[action] = new_q

            y.append(current_qs)
        #Fitting the model to the batch of samples
        #input: State
        #output: Q values with an updated Q value for the action taken in the sample
        updated_model.fit(X, np.array(y), batch_size=BATCH_SIZE, shuffle=False)  

In [205]:
#Trainer

#Stores expirence with enviorment
replay_memory = deque(maxlen = REPLAY_MEMORY_SIZE)

#Initialization
obs, info = env.reset(return_info=True)
prediction_update_couter = 0
atempt = 0

#Allows training to run many times
while atempt < 1000: #can set any number of enviorment runthroughs
    #Retrive the action that should be performed and perfoms it
    action = get_action(np.array(obs).reshape(-1, 4))
    new_obs, reward, done, info = env.step(action)
    
    #Whenever an action is perfromed information of the affect of that action is stored
    replay_memory.append([obs, action, reward, new_obs, done]);
    
    train_model()
    
    #Reset enviorment, decay epsilon, and updated counters when done
    if done:
        obs, info = env.reset(return_info=True)
        
        #epsilon decay
        if epsilon > MIN_EPSILON:
            epsilon *= EPSILON_DECAY
            epsilon = max(MIN_EPSILON, epsilon)
            
        prediction_update_couter += 1
        atempt += 1
    #If not done update the currant state
    else:
        obs = new_obs
    #If we have runthough the enviorment enough times save the model weights and update the prediction model    
    if(prediction_update_couter >= UPDATE_PREDICITON_EVERY):
        updated_model.save_weights('model_weights')
        prediction_model.set_weights(updated_model.get_weights())
        prediction_update_couter = 0


1/1 [==============================] - 0s 14ms/step
[[32.834576 31.321314]]
1/1 [==============================] - 0s 13ms/step
[[32.630146 32.06336 ]]
1/1 [==============================] - 0s 14ms/step
[[32.541485 32.99511 ]]
1/1 [==============================] - 0s 15ms/step
[[32.745472 32.518078]]
1/1 [==============================] - 0s 15ms/step
[[32.615128 33.44111 ]]
1/1 [==============================] - 0s 15ms/step
[[32.772274 32.95506 ]]
1/1 [==============================] - 0s 13ms/step
[[32.781914 32.27622 ]]
1/1 [==============================] - 0s 16ms/step
[[32.478485 33.00816 ]]
1/1 [==============================] - 0s 15ms/step
[[32.4665   32.334923]]
1/1 [==============================] - 0s 15ms/step
[[32.121956 33.077076]]
1/1 [==============================] - 0s 15ms/step
[[32.06719 32.41165]]
1/1 [==============================] - 0s 14ms/step
[[31.87528  31.559795]]
1/1 [==============================] - 0s 15ms/step
[[31.381647 32.130344]]
1/1 [=========

1/1 [==============================] - 0s 23ms/step
[[18.040085 17.99084 ]]
1/1 [==============================] - 0s 13ms/step
[[18.115932 18.566462]]
1/1 [==============================] - 0s 14ms/step
[[18.169447 18.152378]]
1/1 [==============================] - 0s 13ms/step
[[18.120771 18.58724 ]]
1/1 [==============================] - 0s 13ms/step
[[18.268686 18.299032]]
1/1 [==============================] - 0s 13ms/step
[[18.117445 17.632483]]
1/1 [==============================] - 0s 14ms/step
[[18.08855  18.076984]]
1/1 [==============================] - 0s 15ms/step
[[18.041601 18.51292 ]]
1/1 [==============================] - 0s 14ms/step
[[18.189085 18.223083]]
1/1 [==============================] - 0s 15ms/step
[[18.038626 17.556273]]
1/1 [==============================] - 0s 14ms/step
[[18.010618 18.000519]]
1/1 [==============================] - 0s 14ms/step
[[17.980515 18.45553 ]]
1/1 [==============================] - 0s 15ms/step
[[18.113228 18.14613 ]]
1/1 [=======

1/1 [==============================] - 0s 15ms/step
[[19.103287 18.870033]]
1/1 [==============================] - 0s 15ms/step
[[19.145683 19.440891]]
1/1 [==============================] - 0s 14ms/step
[[19.070969 18.904736]]
1/1 [==============================] - 0s 12ms/step
[[19.1104  19.47796]]
1/1 [==============================] - 0s 14ms/step
[[19.031937 18.944   ]]
1/1 [==============================] - 0s 14ms/step
[[19.067183 19.519386]]
1/1 [==============================] - 0s 13ms/step
[[18.899042 19.425543]]
1/1 [==============================] - 0s 15ms/step
[[18.923311 19.03465 ]]
1/1 [==============================] - 0s 14ms/step
[[18.696552 18.32413 ]]
1/1 [==============================] - 0s 15ms/step
[[18.597303 18.72921 ]]
1/1 [==============================] - 0s 14ms/step
[[18.391663 18.032179]]
1/1 [==============================] - 0s 13ms/step
[[18.31149  18.449255]]
1/1 [==============================] - 0s 14ms/step
[[18.12483  17.763523]]
1/1 [=========

1/1 [==============================] - 0s 16ms/step
[[17.740982 17.191353]]
1/1 [==============================] - 0s 15ms/step
[[17.67919  17.608023]]
1/1 [==============================] - 0s 14ms/step
[[17.755322 18.208565]]
1/1 [==============================] - 0s 14ms/step
[[17.78416  17.713692]]
1/1 [==============================] - 0s 12ms/step
[[17.849178 18.306427]]
1/1 [==============================] - 0s 14ms/step
[[17.883135 17.80662 ]]
1/1 [==============================] - 0s 13ms/step
[[17.94135  18.391724]]
1/1 [==============================] - 0s 14ms/step
[[17.981728 17.89367 ]]
1/1 [==============================] - 0s 13ms/step
[[18.03652 18.47555]]
1/1 [==============================] - 0s 14ms/step
[[18.086544 17.98008 ]]
1/1 [==============================] - 0s 14ms/step
[[18.131111 18.56067 ]]
1/1 [==============================] - 0s 15ms/step
[[18.18138  18.070591]]
1/1 [==============================] - 0s 14ms/step
[[18.219498 18.649406]]
1/1 [=========

1/1 [==============================] - 0s 13ms/step
[[18.4934  18.51937]]
1/1 [==============================] - 0s 14ms/step
[[18.441614 17.860561]]
1/1 [==============================] - 0s 13ms/step
[[18.279547 18.248468]]
1/1 [==============================] - 0s 14ms/step
[[18.260712 18.826996]]
1/1 [==============================] - 0s 14ms/step
[[18.35837  18.359108]]
1/1 [==============================] - 0s 14ms/step
[[18.321266 17.702784]]
1/1 [==============================] - 0s 13ms/step
[[18.160578 18.094532]]
1/1 [==============================] - 0s 15ms/step
[[18.142227 18.6747  ]]
1/1 [==============================] - 0s 14ms/step
[[18.244766 18.202902]]
1/1 [==============================] - 0s 14ms/step
[[18.211313 18.77497 ]]
1/1 [==============================] - 0s 14ms/step
[[18.307024 18.297945]]
1/1 [==============================] - 0s 13ms/step
[[18.261675 18.86767 ]]
1/1 [==============================] - 0s 14ms/step
[[18.350897 18.388468]]
1/1 [=========

...

1/1 [==============================] - 0s 13ms/step
[[18.40642 18.97891]]
1/1 [==============================] - 0s 15ms/step
[[18.582829 18.52436 ]]
1/1 [==============================] - 0s 15ms/step
[[18.518444 19.107904]]
1/1 [==============================] - 0s 13ms/step
[[18.674107 18.64324 ]]
1/1 [==============================] - 0s 14ms/step
[[18.585714 19.214739]]
1/1 [==============================] - 0s 13ms/step
[[18.721222 18.741602]]
1/1 [==============================] - 0s 14ms/step
[[18.726845 18.088604]]
1/1 [==============================] - 0s 13ms/step
[[18.503527 18.476027]]
1/1 [==============================] - 0s 14ms/step
[[18.416426 19.045176]]
1/1 [==============================] - 0s 13ms/step
[[18.553865 18.575071]]
1/1 [==============================] - 0s 14ms/step
[[18.559443 17.925468]]
1/1 [==============================] - 0s 13ms/step
[[18.330069 18.308922]]
1/1 [==============================] - 0s 14ms/step
[[18.239668 18.876953]]
1/1 [=========

1/1 [==============================] - 0s 13ms/step
[[18.998186 18.688002]]
1/1 [==============================] - 0s 13ms/step
[[18.908615 19.244352]]
1/1 [==============================] - 0s 13ms/step
[[19.096674 18.804798]]
1/1 [==============================] - 0s 13ms/step
[[18.991598 19.35229 ]]
1/1 [==============================] - 0s 14ms/step
[[19.161966 18.905344]]
1/1 [==============================] - 0s 13ms/step
[[19.033426 19.444317]]
1/1 [==============================] - 0s 13ms/step
[[19.197443 18.997017]]
1/1 [==============================] - 0s 13ms/step
[[19.060314 19.531748]]
1/1 [==============================] - 0s 13ms/step
[[19.214996 19.077713]]
1/1 [==============================] - 0s 13ms/step
[[19.068533 19.605988]]
1/1 [==============================] - 0s 13ms/step
[[19.214636 19.149303]]
1/1 [==============================] - 0s 14ms/step
[[19.054796 19.671455]]
1/1 [==============================] - 0s 14ms/step
[[19.015566 19.729013]]
1/1 [=======

1/1 [==============================] - 0s 13ms/step
[[18.432049 18.442144]]
1/1 [==============================] - 0s 13ms/step
[[18.496918 17.846037]]
1/1 [==============================] - 0s 13ms/step
[[18.255795 18.223766]]
1/1 [==============================] - 0s 13ms/step
[[18.13762  18.778112]]
1/1 [==============================] - 0s 14ms/step
[[18.33172  18.360907]]
1/1 [==============================] - 0s 14ms/step
[[18.395313 17.763117]]
1/1 [==============================] - 0s 14ms/step
[[18.231611 17.544018]]
1/1 [==============================] - 0s 14ms/step
[[17.962276 17.908665]]
1/1 [==============================] - 0s 19ms/step
[[17.834778 18.462286]]
1/1 [==============================] - 0s 14ms/step
[[18.030224 18.048594]]
1/1 [==============================] - 0s 14ms/step
[[18.099167 17.455986]]
1/1 [==============================] - 0s 13ms/step
[[17.874521 17.843256]]
1/1 [==============================] - 0s 14ms/step
[[17.799374 18.416325]]
1/1 [=======

1/1 [==============================] - 0s 14ms/step
[[18.347822 17.752928]]
1/1 [==============================] - 0s 13ms/step
[[18.136953 18.134516]]
1/1 [==============================] - 0s 13ms/step
[[18.067911 18.699196]]
1/1 [==============================] - 0s 13ms/step
[[18.293028 18.29173 ]]
1/1 [==============================] - 0s 15ms/step
[[18.231216 18.855637]]
1/1 [==============================] - 0s 12ms/step
[[18.456324 18.444239]]
1/1 [==============================] - 0s 13ms/step
[[18.389029 19.004671]]
1/1 [==============================] - 0s 14ms/step
[[18.610811 18.592192]]
1/1 [==============================] - 0s 14ms/step
[[18.53441 19.14572]]
1/1 [==============================] - 0s 15ms/step
[[18.74744  18.730957]]
1/1 [==============================] - 0s 13ms/step
[[18.658173 19.276997]]
1/1 [==============================] - 0s 14ms/step
[[18.858097 18.857176]]
1/1 [==============================] - 0s 13ms/step
[[18.756254 19.39686 ]]
1/1 [=========

1/1 [==============================] - 0s 14ms/step
[[18.724487 18.699936]]
1/1 [==============================] - 0s 13ms/step
[[18.622072 19.234377]]
1/1 [==============================] - 0s 14ms/step
[[18.813854 18.817915]]
1/1 [==============================] - 0s 14ms/step
[[18.876705 18.226025]]
1/1 [==============================] - 0s 13ms/step
[[18.527323 19.10793 ]]
1/1 [==============================] - 0s 13ms/step
[[18.72889 18.70098]]
1/1 [==============================] - 0s 14ms/step
[[18.601238 19.222334]]
1/1 [==============================] - 0s 14ms/step
[[18.78632  18.811188]]
1/1 [==============================] - 0s 14ms/step
[[18.838385 18.22038 ]]
1/1 [==============================] - 0s 13ms/step
[[18.575674 18.561705]]
1/1 [==============================] - 0s 13ms/step
[[18.411703 19.074043]]
1/1 [==============================] - 0s 4ms/step - loss: 0.0418 - accuracy: 0.8438


In [199]:
def get_best_action(obs):
    q_vals = updated_model.predict(obs, verbose = 0)
    return np.argmax(q_vals)

In [207]:
#Run the currant model with best actions
obs, info = env.reset(return_info=True)
done = False
score = 0
while not done:
    obs, reward, done, info = env.step(get_best_action(np.array(obs).reshape(-1, 4)))
    score += reward
    env.render()
print(score)

500.0


In [170]:
len(replay_memory)

4837

In [173]:
epsilon

0.025817827831345935